In [1]:
import pandas as pd
type = "H3N2"

In [137]:
import re

df = pd.read_excel(f'data/{type}/{type}_HI_titre.xlsx')
def custom_date_parser(date_str):
    if pd.notna(date_str): 
        if re.match(r'\d{4}_\d+', str(date_str)):
 
            return pd.to_datetime(str(date_str), format='%Y_%m', errors='coerce')
        elif re.match(r'\d{4}', str(date_str)):

            return pd.to_datetime(str(date_str), format='%Y', errors='coerce')
    return pd.NaT  
df['who report'] = df['who report'].apply(custom_date_parser)
# mask = (df['who report'].dt.month == 9)
# df = df[~mask]
df

,virusIsoalte,virusStrain,virusStrain genetic group,CollectionDate,virusstrain passage history,serumIsolate,serumStrain passage,serum ferret number,serumstrain genetic group,serumStrain,HI titre,source,who report,other information,Unnamed: 14
0,A/Panama/2007/99,A/Panama/2007/1999,NaN,NaN,NaN,A/Pan/2007/99,,,,A/Panama/2007/1999,5120,Table 5. Antigenic analyses of influenza A H3N...,2003-01-01,NaN,NaN
1,A/Panama/2007/99,A/Panama/2007/1999,NaN,NaN,NaN,A/NY/55/01,,,,A/New York/55/2001,1280,Table 5. Antigenic analyses of influenza A H3N...,2003-01-01,NaN,NaN
2,A/Panama/2007/99,A/Panama/2007/1999,NaN,NaN,NaN,A/HK/1550/02,,,,A/Hong Kong/1550/2002,5120,Table 5. Antigenic analyses of influenza A H3N...,2003-01-01,NaN,NaN
3,A/Panama/2007/99,A/Panama/2007/1999,NaN,NaN,NaN,A/Egypt/130/02,,,,A/Egypt/130/2002,2560,Table 5. Antigenic analyses of influenza A H3N...,2003-01-01,NaN,NaN
4,A/Panama/2007/99,A/Panama/2007/1999,NaN,NaN,NaN,A/Fuj/411/02,,,,A/Fujian/411/2002,80,Table 5. Antigenic analyses of influenza A H3N...,2003-01-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106944,A/Catalonia/NSVH161512067/2022,A/Catalonia/NSVH161512067/2022,2a.1b,NaN,NaN,A/Norway/24873/21,SIAT,F10/22,2a.3,A/Norway/24873/2021,80,Table H3-14. Antigenic analyses of influenza A...,2023-02-01,NaN,summary
106945,A/Catalonia/NSVH161512067/2022,A/Catalonia/NSVH161512067/2022,2a.1b,NaN,NaN,A/Norway/24873/21,Egg,F11/22,2a.3,A/Norway/24873/2021,160,Table H3-14. Antigenic analyses of influenza A...,2023-02-01,NaN,summary
106946,A/Catalonia/NSVH161512067/2022,A/Catalonia/NSVH161512067/2022,2a.1b,NaN,NaN,A/Poland/97/22,SIAT,F39/22,2a.2,A/Poland/97/2022,320,Table H3-14. Antigenic analyses of influenza A...,2023-02-01,NaN,summary
106947,A/Catalonia/NSVH161512067/2022,A/Catalonia/NSVH161512067/2022,2a.1b,NaN,NaN,A/Slov/8720/2022,SIAT,F24/22,2a.1,A/Slovenia/8720/2022,640,Table H3-14. Antigenic analyses of influenza A...,2023-02-01,NaN,summary


In [138]:

df['HI titre'] = df['HI titre'].astype(str)
df['HI titre'] = df['HI titre'].apply(lambda x: int(x) if x.isnumeric() else (int(x[1:]) if x.startswith('>') else None))
df = df.dropna(subset=['HI titre'])
df = df[df['HI titre'] > 80]


In [139]:
columns = ["virusStrain","serumStrain","who report"]
df = df[columns]
df["virusStrain"] = df["virusStrain"].str.replace(" ","_")
df["serumStrain"] = df["serumStrain"].str.replace(" ","_")
df = df.rename(columns={'who report': 'year'})
df['year'] =pd.to_datetime(df['year']).dt.year
df = df[df['year'] < 2023]
df['CollectionYear'] = df['virusStrain'].str.extract(r'(\d{4}$)')
df['CollectionYear'] = df['CollectionYear'].fillna('0')
df

,virusStrain,serumStrain,year,CollectionYear
0,A/Panama/2007/1999,A/Panama/2007/1999,2003,1999
1,A/Panama/2007/1999,A/New_York/55/2001,2003,1999
2,A/Panama/2007/1999,A/Hong_Kong/1550/2002,2003,1999
3,A/Panama/2007/1999,A/Egypt/130/2002,2003,1999
5,A/Panama/2007/1999,A/Sendai/4952/2002,2003,1999
...,...,...,...,...
101835,A/Bourgoin/758/2022,A/Slovenia/8720/2022,2022,2022
101837,A/Lyon/271/2022,A/Denmark/3264/2019,2022,2022
101838,A/Lyon/271/2022,A/Hong_Kong/2671/2019,2022,2022
101839,A/Lyon/271/2022,A/Cambodia/925256/2020,2022,2022


In [140]:
df.to_excel(f'data/{type}/{type}_titre.xlsx', index=False)

In [141]:
df2 = pd.read_excel(f'data/{type}/{type}_CVVs.xlsx')
mask = (df2['year'] >= 2003) | (df2['S/N'] != 'S')
df2 = df2[mask]
columns = ["CVVs","year","vseq"]
df2 = df2[columns]

In [142]:
df2

,CVVs,year,vseq
0,A/Darwin/6/2021,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
1,A/Darwin/9/2021,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
2,A/Darwin/6/2021,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
3,A/Darwin/9/2021,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
4,A/Cambodia/e0826360/2020,2021,MKTIIALSYILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
...,...,...,...
87,A/Fujian/411/2002,2004,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVPNGTIVKTIT...
88,A/Panama/2007/1999,2003,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
89,A/Panama/2007/1999,2002,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
91,A/Panama/2007/1999,2001,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...


In [143]:

result = df.merge(df2, left_on=["serumStrain", "year"], right_on=["CVVs", "year"], how="inner")
print(result)
result['year'] = result['year'].astype(int)
result['CollectionYear'] = result['CollectionYear'].astype(int)
result = result[(result['year'] - result['CollectionYear']) <= 1]
result = result.drop_duplicates()

# merged_table.to_csv('result.csv', index=False)

                     virusStrain         serumStrain  year CollectionYear  \
0             A/Panama/2007/1999  A/Panama/2007/1999  2003           1999   
1             A/Panama/2007/1999  A/Panama/2007/1999  2003           1999   
2             A/New_York/55/2001  A/Panama/2007/1999  2003           2001   
3             A/New_York/55/2001  A/Panama/2007/1999  2003           2001   
4          A/Hong_Kong/1550/2002  A/Panama/2007/1999  2003           2002   
...                          ...                 ...   ...            ...   
18217  A/Antananarivo/04650/2022     A/Darwin/9/2021  2022           2022   
18218  A/Antananarivo/05015/2022     A/Darwin/9/2021  2022           2022   
18219  A/Antananarivo/05015/2022     A/Darwin/9/2021  2022           2022   
18220        A/Bourgoin/758/2022     A/Darwin/9/2021  2022           2022   
18221        A/Bourgoin/758/2022     A/Darwin/9/2021  2022           2022   

                     CVVs                                               vse

In [144]:
result

,virusStrain,serumStrain,year,CollectionYear,CVVs,vseq
4,A/Hong_Kong/1550/2002,A/Panama/2007/1999,2003,2002,A/Panama/2007/1999,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
6,A/Egypt/130/2002,A/Panama/2007/1999,2003,2002,A/Panama/2007/1999,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
8,A/Sendai/4952/2002,A/Panama/2007/1999,2003,2002,A/Panama/2007/1999,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
10,A/Finland/170/2003,A/Panama/2007/1999,2003,2003,A/Panama/2007/1999,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
12,A/Netherlands/369/2002,A/Panama/2007/1999,2003,2002,A/Panama/2007/1999,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
...,...,...,...,...,...,...
18212,A/La_Reunion/R22.317.55/2022,A/Darwin/9/2021,2022,2022,A/Darwin/9/2021,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
18214,A/Lyon/CHU-R22.264.84/2022,A/Darwin/9/2021,2022,2022,A/Darwin/9/2021,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
18216,A/Antananarivo/04650/2022,A/Darwin/9/2021,2022,2022,A/Darwin/9/2021,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
18218,A/Antananarivo/05015/2022,A/Darwin/9/2021,2022,2022,A/Darwin/9/2021,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...


In [145]:
allSeq =pd.read_excel(f'data/{type}/{type}_all.xlsx')
result = result.merge(allSeq, left_on="virusStrain", right_on="strain", how="left")
result = result.dropna(subset=['seq'])
columns = ["virusStrain","CVVs","year","CollectionYear","seq","vseq"]
result = result[columns]
result = result.rename(columns={'virusStrain': 'Strain',"CVVs":"VVs","year":"ReportYear"})
result = result.drop_duplicates()

In [146]:
result

,Strain,VVs,ReportYear,CollectionYear,seq,vseq
0,A/Hong_Kong/1550/2002,A/Panama/2007/1999,2003,2002,QKLPGNDNSTATLCLGHHAVPNGTLVKTITNDQIEVTNATELVQSS...,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
1,A/Egypt/130/2002,A/Panama/2007/1999,2003,2002,QKLPGNDNSTATLCLGHHAVPNGTLVKTITNDQIEVTNATELVQSS...,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
3,A/Finland/170/2003,A/Panama/2007/1999,2003,2003,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVPNGTLVKTIT...,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
9,A/Latvia/1506/2003,A/Panama/2007/1999,2003,2003,QKLPGNDNSTATLCLGHHAVPNGTLVKTITNDQIEVTNATELVQSS...,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
28,A/Finland/300/2003,A/Panama/2007/1999,2003,2003,QKLPGNDNSTATLCLGHHAVPNGTIVKTITNDQIEVTNATELVQSS...,MKTIIALSYILCLVFAQKLPGNDNSTATLCLGHHAVSNGTLVKTIT...
...,...,...,...,...,...,...
7220,A/Antsirabe/03799/2022,A/Darwin/9/2021,2022,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
7221,A/Antananarivo/04020/2022,A/Darwin/9/2021,2022,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
7225,A/Antananarivo/04650/2022,A/Darwin/9/2021,2022,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...
7226,A/Antananarivo/05015/2022,A/Darwin/9/2021,2022,2022,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...


In [147]:
target_length = 566
def normalize_sequence(row):
    seq = row['seq']
    vseq = row['vseq']
    if len(seq) > target_length:
        return seq[:target_length]
    elif len(seq) < target_length:
        startSeq = seq[:3]
        startIndex = vseq.find(startSeq)
        if startIndex!= -1:
            seq = vseq[:startIndex] + seq
        l = len(seq)
        seq = seq + vseq[l:]

        return seq
    else:
        return seq


result['seq'] = result.apply(normalize_sequence, axis=1)
all_lengths_are_566 = all(len(seq) == target_length for seq in result['seq'])



if not all_lengths_are_566:
    df_non_566 = result[result['seq'].str.len() != target_length]
    print(" seq len !=566：")
    print(df_non_566)


所有序列的长度都为566。


In [148]:
result.drop(['vseq'], axis=1, inplace=True)
result = result[result['seq'].str.startswith('M')]
result.to_excel(f'data/{type}/{type}_preDom.xlsx', index=False)